In [5]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from sklearn.metrics import root_mean_squared_error

In [2]:
df = pd.read_csv('homework_2.1.csv')
df.head()

,Unnamed: 0,time,G1,G2,G3
0,0,0,0.882026,1.441575,0.065409
1,1,1,0.210079,-0.163880,0.140310
2,2,2,0.509369,-0.115242,0.819830
3,3,3,1.150447,1.014698,0.607632
4,4,4,0.973779,-0.046562,0.610066


In [7]:
long_df = df.melt(id_vars='time', value_vars=['G1', 'G2', 'G3'],
                  var_name='group', value_name='outcome')

# Run OLS regression with group fixed effects
model = sm.ols('outcome ~ time + C(group)', data=long_df).fit()

# Display regression summary
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                outcome   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     44.55
Date:                Sun, 01 Jun 2025   Prob (F-statistic):           8.72e-24
Time:                        19:45:00   Log-Likelihood:                -216.89
No. Observations:                 300   AIC:                             441.8
Df Residuals:                     296   BIC:                             456.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.0786      0.071      1.112      0.267      -0.060       0.218
C(group)[T.G2]     0.5111      0.071      7.200      0.000       0.371       0.651
C(group)[T.G3]     0.1905      0.071      2.683      0.008       0.051       0.330
time               0.0090      0.001      8.982      0.000       0.007       0.011
==============================================================================
Omnibus:                        2.113   Durbin-Watson:                   2.062
Prob(Omnibus):                  0.348   Jarque-Bera (JB):                1.817
Skew:                           0.066   Prob(JB):                        0.403
Kurtosis:                       2.642   Cond. No.                         198.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
import statsmodels.api as sm

In [17]:
# Load your data
df = pd.read_csv("homework_2.2.csv")

# Number of bootstrap samples
n_iterations = 1000

# Store the bootstrap samples in a list
bootstrap_samples = []

for _ in range(n_iterations):
    sample = df.sample(frac=1, replace=True)
    bootstrap_samples.append(sample.reset_index(drop=True))  # Keep consistent indexing

# Now you have 1000 full resamples of the original dataset
# Example: Access the first bootstrap sample
sample_1 = bootstrap_samples[0]
print(sample_1.head())

   Unnamed: 0  X         Y         Z
0        7300  1  4.516472  0.385913
1        7232  0  8.369331  1.109147
2        1689  0  6.247470 -1.335559
3        4030  0  1.884727  0.445713
4        6427  0  3.051743  0.606442


In [19]:
treated = df[df['X'] == 1]['Y']
untreated = df[df['X'] == 0]['Y']

# Compute the mean effect
mean_effect = treated.mean() - untreated.mean()

print(f"Mean treatment effect: {mean_effect:.3f}")

Mean treatment effect: 2.921


In [20]:
df = pd.read_csv("homework_2.2.csv")

# Replace with actual column names
treatment_col = 'X'   # e.g., 0 = untreated, 1 = treated
outcome_col = 'Y'       # the variable whose change you're measuring

# Bootstrap parameters
n_iterations = 1000
treatment_effects = []

# Bootstrap loop
for _ in range(n_iterations):
    sample = df.sample(frac=1, replace=True)
    treated = sample[sample[treatment_col] == 1][outcome_col]
    untreated = sample[sample[treatment_col] == 0][outcome_col]
    
    effect = treated.mean() - untreated.mean()
    treatment_effects.append(effect)

# Estimate variance of the bootstrap treatment effects
variance = np.var(treatment_effects, ddof=1)  # Sample variance
print(f"Bootstrap variance of treatment effect: {variance:.6f}")

Bootstrap variance of treatment effect: 0.031854


In [22]:
from scipy.stats import skew

In [23]:
n_iterations = 1000
treatment_effects = []

# Perform bootstrap sampling and run regression Y ~ X
for _ in range(n_iterations):
    sample = df.sample(frac=1, replace=True)
    X_boot = sm.add_constant(sample["X"])
    y_boot = sample["Y"]
    model = sm.OLS(y_boot, X_boot).fit()
    treatment_effects.append(model.params["X"])

# Compute skewness of the bootstrap treatment effect estimates
treatment_skewness = skew(treatment_effects)
treatment_skewness

np.float64(0.07958004464486507)